In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier  # MLP is an NN
from sklearn import svm
import numpy as np
import argparse
import imutils  # If you are unable to install this library, ask the TA; we only need this in extract_hsv_histogram.
import cv2
import os
import random


# Depending on library versions on your system, one of the following imports 
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split

import joblib

In [2]:
path_to_dataset = "/mnt/c/Users/Abdelrahman Tarek/Downloads/Dataset_Updated/data"
target_img_size = (32, 32) # fix image size because classification algorithms THAT WE WILL USE HERE expect that

# We are going to fix the random seed to make our experiments reproducible 
# since some algorithms use pseudorandom generators
random_seed = 42  
random.seed(random_seed)
np.random.seed(random_seed)

In [3]:
def extract_hsv_histogram(img):
    """
    TODO
    1. Resize the image to target_img_size using cv2.resize
    2. Convert the image from BGR representation (cv2 is BGR not RGB) to HSV using cv2.cvtColor
    3. Acquire the histogram using the cv2.calcHist. Apply the functions on the 3 channels. For the bins 
        parameter pass (8, 8, 8). For the ranges parameter pass ([0, 180, 0, 256, 0, 256]). Name the histogram
        <hist>.
    """
    
    img = cv2.resize(img, target_img_size)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    hist = cv2.calcHist([img], [3], None, [8,8,8], [0, 180, 0, 256, 0, 256])
    
    if imutils.is_cv2():
        hist = cv2.normalize(hist)
    else:
        cv2.normalize(hist, hist)
    return hist.flatten()   

In [4]:
def extract_hog_features(img):
    """
    TODO
    You won't implement anything in this function. You just need to understand it 
    and understand its parameters (i.e win_size, cell_size, ... etc)
    """
    img = cv2.resize(img, target_img_size)
    win_size = (32, 32)
    cell_size = (4, 4)
    block_size_in_cells = (2, 2)
    
    block_size = (block_size_in_cells[1] * cell_size[1], block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0])
    nbins = 9  # Number of orientation bins
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    h = hog.compute(img)
    h = h.flatten()
    return h.flatten()

In [5]:
def extract_raw_pixels(img):
    """
    TODO
    The classification algorithms we are going to use expect the input to be a vector not a matrix. 
    This is because they are general purpose and don't work only on images.
    CNNs, on the other hand, expect matrices since they operate on images and exploit the 
    arrangement of pixels in the 2-D space.
    
    So, what we only need to do in this function is to resize and flatten the image.
    """
    return cv2.resize(np.array(img).flatten(), target_img_size)

In [6]:
def extract_features(img, feature_set='hog'):
    """
    TODO
    Given either 'hsv_hist', 'hog', 'raw', call the respective function and return its output
    """
    if feature_set == 'hog':
        return extract_hog_features(img)
    elif feature_set == 'hsv_hist':
        extract_hsv_histogram(img)
    else:
        extract_raw_pixels(img)

In [7]:
def load_dataset(feature_set='hog'):
    features = []
    labels = []
    img_filenames = os.listdir(path_to_dataset)

    for i, fn in enumerate(img_filenames):
        if fn.split('.')[-1] != 'png':
            continue

        label = fn.split('.')[0]
        labels.append(label)

        path = os.path.join(path_to_dataset, fn)
        img = cv2.imread(path)
        features.append(extract_features(img, feature_set))
        
        # show an update every 1,000 images
        if i > 0 and i % 1000 == 0:
            print("[INFO] processed {}/{}".format(i, len(img_filenames)))
        
    return features, labels  

In [8]:
# TODO understand the hyperparameters of each classifier
classifiers = {
    #'SVM': svm.LinearSVC(random_state=random_seed),
    #'KNN': KNeighborsClassifier(n_neighbors=7),
    'NN': MLPClassifier(solver='adam', random_state=random_seed, hidden_layer_sizes=(500,100,50), max_iter=50, verbose=1)
}

In [9]:
# This function will test all our classifiers on a specific feature set
def run_experiment(feature_set):
    
    # Load dataset with extracted features
    print('Loading dataset. This will take time ...')
    features, labels = load_dataset(feature_set)
    print('Finished loading dataset.')
    
    # Since we don't want to know the performance of our classifier on images it has seen before
    # we are going to withhold some images that we will test the classifier on after training 
    train_features, test_features, train_labels, test_labels = train_test_split(
        features, labels, test_size=0.2, random_state=random_seed)
    
    for model_name, model in classifiers.items():
        print('############## Training', model_name, "##############")
        # Train the model only on the training features
        model.fit(train_features, train_labels)
        
        # Test the model on images it hasn't seen before
        accuracy = model.score(test_features, test_labels)
        
        print(model_name, 'accuracy:', accuracy*100, '%')

In [10]:
run_experiment('hog')

Loading dataset. This will take time ...
[INFO] processed 1000/60817
[INFO] processed 2000/60817
[INFO] processed 3000/60817
[INFO] processed 4000/60817
[INFO] processed 5000/60817
[INFO] processed 6000/60817
[INFO] processed 7000/60817
[INFO] processed 8000/60817
[INFO] processed 9000/60817
[INFO] processed 10000/60817
[INFO] processed 11000/60817
[INFO] processed 12000/60817
[INFO] processed 13000/60817
[INFO] processed 14000/60817
[INFO] processed 15000/60817
[INFO] processed 16000/60817
[INFO] processed 17000/60817
[INFO] processed 18000/60817
[INFO] processed 19000/60817
[INFO] processed 20000/60817
[INFO] processed 21000/60817
[INFO] processed 22000/60817
[INFO] processed 23000/60817
[INFO] processed 24000/60817
[INFO] processed 25000/60817
[INFO] processed 26000/60817
[INFO] processed 27000/60817
[INFO] processed 28000/60817
[INFO] processed 29000/60817
[INFO] processed 30000/60817
[INFO] processed 31000/60817
[INFO] processed 32000/60817
[INFO] processed 33000/60817
[INFO] proc

In [11]:
for model_name, model in classifiers.items():
    filename = '/mnt/d/Projects/PYTHON/MusicSheetReader/music_notes_' + model_name + '.joblib.pkl'
    _ = joblib.dump(model, filename, compress=9)